In [23]:
https://www.analyticsvidhya.com/blog/2024/02/fine-tuning-a-tiny-llama-model-with-unsloth/

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-23-ea160fedd2c0>, line 1)

In [24]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install wandb
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
pass

In [25]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit", # "unsloth/tinyllama" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/config.json
Model config LlamaConfig {
  "_name_or_path": "unsloth/tinyllama-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_

==((====))==  Unsloth: Fast Llama patching release 2024.2
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/config.json
Model config LlamaConfig {
  "_name_or_path": "unsloth/tinyllama-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_

In [26]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = True, # @@@ IF YOU GET OUT OF MEMORY - set to True @@@
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import pickle
DATA_PATH = "/content/drive/My Drive"
infile = open(DATA_PATH+'/combined_df.pkl','rb')
df = pickle.load(infile)

In [30]:
df = df.rename(columns={'Email Text': 'input', 'Email Type': 'output'})
df

,input,output
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Malicious Email
4,software at incredibly low prices ( 86 % lower...,Malicious Email
...,...,...
23816,Subject: put the 10 on the ft\r\nthe transport...,Safe Email
23817,Subject: 3 / 4 / 2000 and following noms\r\nhp...,Safe Email
23818,Subject: calpine daily gas nomination\r\n>\r\n...,Safe Email
23819,Subject: industrial worksheets for august 2000...,Safe Email


In [31]:
class_counts = df['output'].value_counts()

In [32]:
class_counts

Safe Email         14994
Malicious Email     8827
Name: output, dtype: int64

In [33]:
trimmed_df = df.sample(n=10000, random_state=3407)

In [34]:
trimmed_df

,input,output
6853,"re : lloyd , i think that we can arrange a few...",Safe Email
9872,WE NEED HELP. We are a 14 year old fortune 50...,Malicious Email
15377,"slipped ambl 1 en , xanaax , tussioneex , \ / ...",Malicious Email
10540,"The pay's not good (9,300 a year), but there i...",Safe Email
9718,vacation scheduled i will be on vacation frida...,Safe Email
...,...,...
16575,= ? utf - 8 ? q ? you might become to ? = = ? ...,Malicious Email
15772,honored by two keynote speakers international ...,Safe Email
14573,exactseek - verify your site submission the fo...,Malicious Email
17171,enron capitalism a primer on how to succeed in...,Safe Email


In [35]:
class_counts = trimmed_df['output'].value_counts()
class_counts

Safe Email         6268
Malicious Email    3732
Name: output, dtype: int64

In [36]:
# Convert from df to dataset
import datasets
dataset = datasets.Dataset.from_pandas(trimmed_df)

In [37]:
dataset

Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 10000
})

In [38]:
# @title prepare data

alpaca_prompt = """.

Given the input email and the reponse determine if the email provided below is safe or potentially malicious based on its
content. If the email seems legitimate and does not contain any
harmful intent, threats, or suspicious requests, classify it as
"safe". If the email contains elements typical of phishing attempts,
such as requests for sensitive information, suspicious links, or unusual
sender addresses, classify it as "potentially malicious".

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [39]:
dataset[0]
dataset_dict = dataset.train_test_split(test_size=0.01)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['input', 'output', '__index_level_0__', 'text'],
        num_rows: 9900
    })
    test: Dataset({
        features: ['input', 'output', '__index_level_0__', 'text'],
        num_rows: 100
    })
})

In [40]:
# @title wandb init
import wandb
wandb.login()

True

In [41]:
import os
%env WANDB_WATCH=all
%env WANDB_SILENT=true
os.environ.get("WANDB_SILENT")

env: WANDB_WATCH=all
env: WANDB_SILENT=true


'true'

In [42]:
import os

from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.utils import logging
import wandb

logging.set_verbosity_info()
project_name = "tiny-llama"
entity = "wandb"
# os.environ["WANDB_LOG_MODEL"] = "checkpoint"

wandb.init(project=project_name, name = "run_10000")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Packs short sequences together to save time!
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps = 4,
        evaluation_strategy="steps",
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb",  # enable logging to W&B
        # run_name="tiny-llama-alpaca-run6",  # name of the W&B run (optional)
        logging_steps=1,  # how often to log to W&B
        logging_strategy = 'steps',
        save_total_limit=2,
    ),
)

using `logging_steps` to initialize `eval_steps` to 1
PyTorch: setting up devices


Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (8311 > 2048). Running this sequence through the model will result in indexing errors


Generating train split: 0 examples [00:00, ? examples/s]

Using auto half precision backend


In [43]:
trainer.eval_dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 27
})

In [44]:
trainer_stats = trainer.train()
wandb.finish()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,010 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 376
 "-____-"     Number of trainable parameters = 25,231,360
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
1,2.513300,2.456930
2,2.688200,2.456757
3,2.455400,2.456362
4,2.432500,2.455521
5,2.722200,2.454160
6,2.417100,2.452044
7,2.513800,2.449020
8,2.542800,2.444890
9,2.341100,2.439451
10,2.465100,2.432946


***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 27
  Batch size = 2
***** Running Evaluation *****
  Num examples = 

eval/loss,██▆▅▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▇▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▇▁▁
eval/samples_per_second,█████████████▂██▂██▂██████████████▁██▂██
eval/steps_per_second,█████████████▂██▂██▂██████████████▁██▂██
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,█▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▂▃▅▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,▆█▅▅▅▅▄▃▄▃▃▃▂▂▂▄▃▂▄▂▃▃▃▃▄▁▂▁▃▁▃▂▄▅▅▃▁▃▆▂
train/total_flos,▁
train/train_loss,▁


In [48]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/config.json
Model config LlamaConfig {
  "_name_or_path": "unsloth/tinyllama",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit":

In [46]:
from google.colab import files
files.download('lora_model')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
model_path = '/content/drive/My Drive/lora_model'
model.save(model_path)

AttributeError: 'LlamaForCausalLM' object has no attribute 'save'

In [52]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Is the following an phishing attempt?", # instruction
        """Subject: PayPal Payment Not Processed Successfully

Personal Assistant Needed
You are invited to participate in a Part-time work offer for current
staff/students. For more information About the task CLICK HERE
Application will be received and you will get a response between
24- 48 hours.

        """, # input
        "", # output - leave this blank for a generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> .\n\nGiven the input email and the reponse determine if the email provided below is safe or potentially malicious based on its\ncontent. If the email seems legitimate and does not contain any\nharmful intent, threats, or suspicious requests, classify it as\n"safe". If the email contains elements typical of phishing attempts,\nsuch as requests for sensitive information, suspicious links, or unusual\nsender addresses, classify it as "potentially malicious".\n\n### Input:\nIs the following an phishing attempt?\n\n### Response:\nSubject: PayPal Payment Not Processed Successfully\n\nPersonal Assistant Needed\nYou are invited to participate in a Part-time work offer for current\nstaff/students. For more information About the task CLICK HERE\nApplication will be received and you will get a response between\n24- 48 hours.\n\n        \n\n\n### Input:\nSubject: Re: PayPal Payment Not Processed Successfully\n\nPersonal Assistant Needed\nYou are invited to participate in a Part-time work off